In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

# Sellers: Coding olistdash/seller.py

The goal is to find Sellers that repeatedly underperform vs others, and understand why.  
This will help us shape our recommendations on how to improve Olist's profit margin.

## 1 - Coding `olistdash/seller.py` 

- Creating the method `get_training_data` in `olistdash/seller.py` that will return the following DataFrame:

  - `seller_id` (_str_) _the id of the seller **UNIQUE**
  - `seller_city` (_str_) _the city where seller is located_
  - `seller_state` (_str_) _the state where seller is located_
  - `delay_to_carrier` (_float_) _Average delay_to_carrier per seller. Return 0 if the order is delivered before the shipping_limit_date, otherwise the absolute value of the delay
  - `wait_time` (_float_) _Average wait_time (duration of deliveries) per seller_
  - `date_first_sale` (_datetime_) _Date of first sales on Olist_
  - `date_last_sale` (_datetime_) _Date of last sales on Olist_
  - `share_of_five_stars` (_float_) _The share of five star orders for orders in which the seller was involved_
  - `share_of_one_stars` (_float_) _The share of one star orders for orders in which the seller was involved_
  - `review_score` (_float_) _The average review score for orders in which the seller was involved_
  - `n_orders` (_int_) _The number of unique orders the seller was involved with._
  - `quantity` (_int_) _The total number of items sold by this seller_
  - `quantity_per_order`(_float_) _The mean number of items per order for this seller_
  - `sales` (_float_) _The total sales associated with this seller (excluding freight value)_ in BRL

### `get_seller_features`
Returns a DataFrame with: 'seller_id', 'seller_city', 'seller_state'

In [1]:
def get_seller_features(self):
    """
        Returns a DataFrame with:
        'seller_id','seller_city', 'seller_state'
        """
    # Making a copy before using inplace=True to avoid modifying
    # self.data
    sellers = self.data['sellers'].copy()
    sellers.drop('seller_zip_code_prefix', axis=1, inplace=True)
    # There are multiple rows per seller
    sellers.drop_duplicates(inplace=True)
    return sellers


### `get_seller_delay_wait_time`
Returns a DataFrame with: 'seller_id', 'delay_to_carrier', 'wait_time'

In [3]:
def get_seller_delay_wait_time(self):
        """
        Returns a DataFrame with:
        'seller_id', 'delay_to_carrier', 'seller_state'
        """
        # Get data
        order_items = self.data['order_items'].copy()
        orders = self.data['orders'].query("order_status=='delivered'").copy()

        ship = order_items.merge(orders, on='order_id')

        # Handling datetime
        ship.loc[:, 'shipping_limit_date'] = pd.to_datetime(
            ship['shipping_limit_date'])
        ship.loc[:, 'order_delivered_carrier_date'] = pd.to_datetime(
            ship['order_delivered_carrier_date'])
        ship.loc[:, 'order_delivered_customer_date'] = pd.to_datetime(
            ship['order_delivered_customer_date'])
        ship.loc[:, 'order_purchase_timestamp'] = pd.to_datetime(
            ship['order_purchase_timestamp'])

        # Computing delay and wait_time
        def handle_early_dropoff(x):
            if x < 0:
                return abs(x)
            return 0

        def delay_to_logistic_partner(df):
            df['delay'] = (
                df.shipping_limit_date -
                df.order_delivered_carrier_date) / np.timedelta64(24, 'h')
            df.loc[:,'delay'] = df.delay.apply(handle_early_dropoff)
            return np.mean(df.delay)

        def order_wait_time(df):
            days = np.mean(
                (df.order_delivered_customer_date - df.order_purchase_timestamp)
                / np.timedelta64(24, 'h')
            )
            return days

        delay = ship\
            .groupby('seller_id')\
                .apply(delay_to_logistic_partner)\
                    .reset_index()

        delay.columns = ['seller_id', 'delay_to_carrier']

        wait = ship\
            .groupby('seller_id')\
                .apply(order_wait_time)\
                    .reset_index()
        wait.columns = ['seller_id', 'wait_time']

        order_wait_time_df = delay.merge(wait, on='seller_id')

        return order_wait_time_df

### `get_active_dates`
Returns a DataFrame with 'seller_id', 'date_first_sale', 'date_last_sale'

In [4]:
def get_active_dates(self):
    """
        This function returns a DataFrame with: 'seller_id', 'date_first_sale',
        'date_last_sale'
        """
    orders = self.data['orders'][['order_id', 'order_approved_at']].copy()

    # creating two new columns with a view to aggregate
    orders.loc[:,
               'date_first_sale'] = pd.to_datetime(orders['order_approved_at'])
    orders['date_last_sale'] = orders['date_first_sale']

    return orders.merge(
        self.matching_table[['seller_id', 'order_id']], on="order_id")\
        .groupby('seller_id')\
        .agg({
            "date_first_sale": min,
            "date_last_sale": max
        })


### `get_review_score`
['seller_id', 'share_of_five_stars', 'share_of_one_stars', 'review_score']

In [5]:
def get_review_score(self):
    """
        This function returns a DataFrame with:
        'seller_id', 'share_of_five_stars', 'share_of_one_stars', 'review_score'
        """
    matching_table = self.matching_table
    orders_reviews = self.order.get_review_score()

    # Since the same seller can appear multiple times in the same order,
    # a (seller <> order) matching table has been created.

    matching_table = matching_table[['order_id', 'seller_id']]\
        .drop_duplicates()
    reviews_df = matching_table.merge(orders_reviews, on='order_id')
    reviews_df = reviews_df.groupby('seller_id', as_index=False).agg({
        'dim_is_one_star':
        'mean',
        'dim_is_five_star':
        'mean',
        'review_score':
        'mean'
    })
    # Rename columns
    reviews_df.columns = [
        'seller_id', 'share_of_one_stars', 'share_of_five_stars',
        'review_score'
    ]

    return reviews_df


### `get_quantity`
['seller_id', 'n_orders', 'quantity', 'quantity_per_order']

In [6]:
def get_quantity(self):
    """
        Returns a DataFrame with:
        'seller_id', 'n_orders', 'quantity', 'quantity_per_order'
        """
    order_items = self.data['order_items']

    n_orders = order_items.groupby('seller_id')['order_id']\
        .nunique().reset_index()
    n_orders.columns = ['seller_id', 'n_orders']

    quantity = order_items.groupby('seller_id', as_index=False)\
        .agg({'order_id': 'count'})
    quantity.columns = ['seller_id', 'quantity']

    result = n_orders.merge(quantity, on='seller_id')
    result['quantity_per_order'] = result['quantity'] / result['n_orders']
    return result


### `get_sales`
['seller_id', 'sales']

In [7]:
def get_sales(self):
    """
        Returns a DataFrame with:
        'seller_id', 'sales'
        """
    return self.data['order_items'][['seller_id', 'price']]\
        .groupby('seller_id')\
        .sum()\
        .rename(columns={'price': 'sales'})

### Merging DataFrames created in the functions above.
### `get_training_data`

In [8]:
def get_training_data(self):
    """
        Merging all the DataFrames created before.
        Returns a DataFrame with:
        'seller_id', 'seller_state', 'seller_city', 'delay_to_carrier',
        'wait_time', 'share_of_five_stars', 'share_of_one_stars',
        'seller_review_score', 'n_orders', 'quantity,' 'date_first_sale',
        'date_last_sale', 'sales'
        """

    training_set =\
        self.get_seller_features()\
            .merge(
            self.get_seller_delay_wait_time(), on='seller_id'
           ).merge(
            self.get_active_dates(), on='seller_id'
           ).merge(
            self.get_review_score(), on='seller_id'
           ).merge(
            self.get_quantity(), on='seller_id'
           ).merge(
            self.get_sales(), on='seller_id'
           )

    return training_set


---
### Saving logic into seller.py

Testing the code from seller.py

In [12]:
from olistdash.seller import Seller
data = Seller().get_training_data()

data.head()

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,share_of_one_stars,share_of_five_stars,review_score,n_orders,quantity,quantity_per_order,sales
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,1.514329,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,0.333333,0.333333,3.00,3,3,1.000,218.70
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.155190,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,0.050000,0.725000,4.55,40,41,1.025,11703.07
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.000000,1.000000,5.00,1,1,1.000,158.00
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.000000,1.000000,5.00,1,1,1.000,79.99
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,1.000000,0.000000,1.00,1,1,1.000,167.99
